<h1> DS200A Computer Vision Assignment</h1>

<h2>  Part Four: Neural networks </h2>	


Build a neural network classifier using an architecture of your choosing. This application
of deep learning can be done in PyTorch, TensorFlow, or a framework of your choice. This is the
industry standard for image classification. Describe your network and assess its performance. To
receive extra credit, your neural network classifier must outperform your other methods.



In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib
import skimage
from skimage import data
from skimage import io
import os
import cv2
import torch

In [ ]:
data_original = pd.read_hdf("data.h5", "data")
print(len(data_original))

In [ ]:
data_filtered = data_original.copy()

In [ ]:
# hyperparameters
batch_size = 16
img_w, img_h = 32, 32
num_classes = 20

In [ ]:
def get_shape(x):
    return len(x.shape)
data_filtered['shape'] = data_filtered['Pictures'].apply(get_shape)
data_filtered = data_filtered[data_filtered['shape']==3]
print(len(data_filtered))
data_filtered.head()

In [ ]:
# def transpose_tensor(x):
#     return x.transpose(2,0,1)
# data['Pictures'] = data['Pictures'].apply(transpose_tensor)
# print(len(data))
# data.head()

In [ ]:
# Dataset processing
def load_data(df, img_w, img_h):
    print('Reading training images')
    img_list, label_list = [], []
    for index, data in df.iterrows():
        img = data['Pictures']
        label = data['Encoding']
        #resize each image to the standard img_w * img_h 
        img = cv2.resize(img, (img_w, img_h), cv2.INTER_LINEAR)
        img_list.append(img), label_list.append(label)
    img_list, label_list = np.array(img_list, dtype=np.uint8), np.array( label_list, dtype=np.int32 )
    img_list = img_list.astype('float32')
    label_list = label_list.astype('long')
    img_list = img_list / 255
    return img_list, label_list

data, label = load_data(data_filtered, img_w, img_h)
print(data.shape, label.shape)

idx = np.arange( data.shape[0] )
np.random.shuffle(idx)
data, label = data[idx], label[idx]

In [ ]:
split_ratio = 0.8
split = np.int(data.shape[0]* split_ratio)
x_train, y_train = data[:split].transpose(0,3,1,2), label[:split].reshape(-1)
x_val, y_val =data[split:].transpose(0,3,1,2), label[split:].reshape(-1)
print(x_train.shape, y_train.shape)
print(x_val.shape, y_val.shape)
print(x_train[0])

In [ ]:
from torchvision import transforms

channel1_mean, channel1_std = x_train[:,0,:,:].mean(), x_train[:,0,:,:].std()
channel2_mean, channel2_std = x_train[:,1,:,:].mean(), x_train[:,1,:,:].std()
channel3_mean, channel3_std = x_train[:,2,:,:].mean(), x_train[:,2,:,:].std()

normalize_data = transforms.Compose([
    transforms.Normalize((channel1_mean, channel2_mean, channel3_mean),
                          (channel1_std, channel2_std, channel3_std)),
])    
for i in range(len(x_val)):
    x_val[i] = normalize_data(torch.from_numpy(x_val[i]))
for i in range(len(x_train)):
    x_train[i] = normalize_data(torch.from_numpy(x_train[i]))

print(x_train[0])
print(y_val)

In [ ]:
from torch.utils.data.dataset import Dataset

class ImageDataset(Dataset):
    def __init__(self, image, label):
        self.image = image
        self.label = label
        
    def __getitem__(self, index):
        # stuff
        return (self.image[index], self.label[index])

    def __len__(self):
        return len(self.label)

In [ ]:
train_dataset = ImageDataset(x_train, y_train)
val_dataset = ImageDataset(x_val, y_val)

In [ ]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class QAlexNet(nn.Module):
    def __init__(self, num_classes=10):
        super(QAlexNet, self).__init__()
        self.conv1 = nn.Conv2d(
            3, 64, kernel_size=5, stride=1, padding=2)  # 32x32x3 -> 32x32x64
        # self.pool1=nn.MaxPool2d(kernel_size=3, stride=2, padding =1 )# 32x32x64
        # -> 16x16x64
        self.bn1 = nn.BatchNorm2d(64)
        self.conv2 = nn.Conv2d(
            64, 64, kernel_size=5, stride=1, padding=2)  # 16x16x64 -> 16x16x64
        # self.pool2=nn.MaxPool2d(kernel_size=3, stride=2, padding = 1)# 16x16x64
        # -> 8x8x64
        self.bn2 = nn.BatchNorm2d(64)
        self.fc1 = nn.Linear(64 * 8 * 8, 384) # self.fc1 = nn.Linear(64 * 8 * 8, 384)
        self.fc2 = nn.Linear(384, 192)
        self.fc3 = nn.Linear(192, num_classes)

    def squeeze_layers(self, sl=None):
        for k in self._modules.keys():
            if k in sl:
                for param in self._modules[k].parameters():
                    param.requires_grad = False
                    print(param.requires_grad)

    def back(self):
        for k in self._modules.keys():
            for param in self._modules[k].parameters():
                param.requires_grad = True

    def forward(self, x):
        x = F.max_pool2d(self.bn1(F.relu(self.conv1(x))), 3, 2, 1)
        x = F.max_pool2d(self.bn2(F.relu(self.conv2(x))), 3, 2, 1)
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [ ]:
def one_hot(x, K):
    return np.array(x[:, None] == np.arange(K)[None, :], dtype=int)

def learning_rate_with_decay(
        lr,
        batch_size,
        batch_denom,
        batches_per_epoch,
        boundary_epochs,
        decay_rates):
    initial_learning_rate = lr*1. #* batch_size / batch_denom

    boundaries = [int(batches_per_epoch * epoch) for epoch in boundary_epochs]
    vals = [initial_learning_rate * decay for decay in decay_rates]

    def learning_rate_fn(itr):
        lt = [itr < b for b in boundaries] + [True]
        i = np.argmax(lt)
        return vals[i]

    return learning_rate_fn

def inf_generator(iterable):
    """Allows training with DataLoaders in a single infinite loop:
        for i, (x, y) in enumerate(inf_generator(train_loader)):
    """
    iterator = iterable.__iter__()
    while True:
        try:
            yield iterator.__next__()
        except StopIteration:
            iterator = iterable.__iter__()
            
def accuracy(model, dataset_loader):#, args):
    total_correct = 0
    for x, y in dataset_loader:
        #if args.gpu:
        #    x = x.cuda()
        y = one_hot(np.array(y.numpy()), 20)

        target_class = np.argmax(y, axis=1)
        predicted_class = np.argmax(model(x).cpu().detach().numpy(), axis=1)
        total_correct += np.sum(predicted_class == target_class)
    return total_correct / len(dataset_loader.dataset)

In [ ]:
model = QAlexNet(num_classes=20)
criterion = nn.CrossEntropyLoss()

In [ ]:
data_gen = inf_generator(train_loader)
batches_per_epoch = len(train_loader)

lr_fn = learning_rate_with_decay(
    0.01,
    batch_size,
    batch_denom=128,
    batches_per_epoch=batches_per_epoch,
    boundary_epochs=[30,60],
    decay_rates=[1, 0.1, 0.01]
)

optimizer = torch.optim.SGD(
    model.parameters(),
    lr=0.01,
    momentum=0.9,
    weight_decay=1e-4)

In [ ]:
epoch = 0
iterations = 0
epoches, train_losses, val_accuracies = [], [], []

train_loss = 0.0

print(f'Numer of batches per epoch: {batches_per_epoch}')
while iterations < (100 * batches_per_epoch):

    print(f'Iteration number: {iterations}')
    iterations += 1

    for param_group in optimizer.param_groups:
        param_group['lr'] = lr_fn(iterations)

    optimizer.zero_grad()
    x, y = data_gen.__next__()
#     if args.gpu:
#         x = x.cuda()
#         y = y.cuda()
    logits = model(x)
    loss = criterion(logits, y)
    train_loss += loss

    loss.backward()
    optimizer.step()

    if iterations % batches_per_epoch == 0:
        train_loss /= batches_per_epoch
        epoch += 1

        with torch.no_grad():
            val_acc = accuracy(model, val_loader)#, args)
            train_acc = accuracy(model, train_loader)#, args)
            print(
                "Epoch {:04d} | Val Acc {:.4f} | Train Acc {:.4f} | Training Loss {:.4f}".format(
                    iterations // batches_per_epoch,
                    val_acc, train_acc,
                    train_loss))
        epoches.append(epoch)
        train_losses.append(train_loss)
        val_accuracies.append(val_acc)
        train_loss = 0.
